# `transform.py`

DESC is a [pseudo-spectral](https://en.wikipedia.org/wiki/Pseudo-spectral_method) code, where the dependent variables $R$, $Z$, $\lambda$, as well as parameters such as the plasma boundary and profiles are represented by spectral basis functions.
These parameters are interpolated to a grid of collocation nodes in real space.
See the section on [basis functions](https://desc-docs.readthedocs.io/en/latest/notebooks/basis_grid.html#Basis-functions) for more information.

Representing the parameters as a sum of spectral basis functions simplifies finding solutions to the relavant physics equations.
This is similar to how the Fourier transform reduces a complicated operation like differentiation in real space to multiplication by frequency in the frequency space.
In particular, and a more relavant example, seeking a solution to a partial differential equation as a linear combination of spectral basis functions reduces the PDE to a set of ordinary differential equations of the coefficients which compose that linear combination.
The resulting ODEs can then be solved numerically.

Once it is known which combination of basis functions in the spectral space compose the relavant parameters, such as the plasma boundary etc., these functions in the spectral space need to be transformed back to real space to better understand their behavior in real space.

The `Transform` class provides methods to transform between spectral and real space.
Each `Transform` object contains a spectral basis and a grid.

## `build()` and `transform(c)`

The `build()` method builds the matrices for a particular grid which define the transformation from spectral to real space.
This is done by evaluating the basis at each point of the grid.
Generic examples of this type of transformation are the inverse Fourier transform and a change of basis matrix for fininte dimiensional vector spaces.

The `transform(c)` method applies the resulting matrix to the given vector, $\mathbf{c}$, which specify the coefficients of the basis associated with this `Transform` object.
This transforms the given vector of spectral coefficients to real space values.

The matrices are computed for each derivative order specified when the `Transform` object was constructed.
The highest deriviative order at which to compute the transforms is specified by an array of three integers (one for each coordinate in $\rho, \theta, \zeta$) given as the `derivs` argument.

Define the transform matrix as $A_{(d\rho,d\theta,d\zeta)}$ for the derivative of order ${(d\rho,d\theta,d\zeta)}$ (where each are integers).
This matrix transforms a spectral basis evaluated on a certain grid with a given set of coefficients $\mathbf{c}$ to real space values $x$.

$$ A\mathbf{c} = \mathbf{x}$$

- $\mathbf{c}$ is a vector of length `Transform.basis.num_modes` (the number of modes in the basis)
- $\mathbf{x}$ is a vector of length `Transform.grid.num_nodes` (the number of nodes in the grid)
- $A$ is a matrix of shape `(num_nodes,num_modes)`.

As a simple example, if the basis is a Fourier series given by $f(\zeta) = 2 + 4*cos(\zeta)$, and the grid is $\mathbf{\zeta} =\begin{bmatrix}0\\ \pi\end{bmatrix}$, then

$$\mathbf{c}=\begin{bmatrix} 2\\ 4 \end{bmatrix}$$
$$A_{(0, 0, 0)} = \begin{bmatrix} 1 & cos(0)\\ 1& cos(\pi) \end{bmatrix}$$
$$A_{(0, 0, 0)}\mathbf{c} = \begin{bmatrix} 1& 1\\ 1& -1 \end{bmatrix} \begin{bmatrix} 2\\ 4 \end{bmatrix} = \begin{bmatrix} 6 \\ -2  \end{bmatrix}$$

## `build_pinv()` and `fit(x)`

The `build_pinv` method builds the matrix which defines the [pseudoinverse (Moore–Penrose inverse)](https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_inverse) transformation.
In particular, this is a transformation from real space values to coefficients of a spectral basis.
Generic examples of this type of transformation are the Fourier transform and a change of basis matrix for fininte dimiensional vector spaces.

Any vector of values in real space can be represented as coefficients to some linear combination of a basis in spectral space.
However, the basis of a particular `Transform` may not be able to exactly represent a given vector of real space values.
In that case, the system $A \mathbf{c} = \mathbf{x}$ would be inconsistent.

The `fit(x)` method applies $A^{\dagger}$ to the vector $\mathbf{x}$ of real space values.
This yields the coefficients that best allow the basis of a `Transform` object to approximate $\mathbf{x}$ in spectral space.
The pseudo-inverse transform, $A^{\dagger}$, applied to $\mathbf{x}$ represents the least-squares solution for the unknown given by $\mathbf{c}$ to the system $A \mathbf{c} = \mathbf{x}$.

It is required from the least-squares solution, $A^{\dagger} \mathbf{x}$, that

$$A^{\dagger} \mathbf{x} = \min_{∀ \mathbf{c}} \lvert A \mathbf{c} - \mathbf{x} \rvert \; \text{so that} \; \lvert A A^{\dagger} \mathbf{x} - \mathbf{x}\rvert \; \text{is minimized}$$

For this to be true, $A A^{\dagger}$ must be the orthogonal projection onto the image of the transformation $A$.
It follows that

$$A A^{\dagger} \mathbf{x} - \mathbf{x} ∈ (\text{image}(A))^{\perp} = \text{kernel}(A^T)$$

$$
\begin{align*}
    A^T (A A^{\dagger} \mathbf{x} - \mathbf{x}) &= 0 \\
    A^T A A^{\dagger} \mathbf{x} &= A^T \mathbf{x} \\
    A^{\dagger} &= (A^T A)^{-1} A^{T} \quad \text{if} \; A \; \text{is invertible}
\end{align*}
$$

Equivalently, if $A = U S V^{T}$ is the singular value decomposition of the transform matrix $A$, then

$$ A^{\dagger} = V S^{+} U^{T}$$

where the diagonal of $S^{+}$ has entries which are are the recipricols of the entries on the diagonal of $S$, except that any entries in the diagonal with $0$ for the singular value are kept as $0$.
(If there are no singular values corresponding to $0$, then $S^{+}=S^{-1} \implies A^{\dagger}=A^{-1}$, and hence $A^{-1}$ exists because there are no eigenvectors with eigenvalue $0^{2}$).

## Transform build options
There are three different options from which the user can choose to build the transform matrix and its pseudoinverse.

### Option 1: `direct1`

With this option, the transformation matrix is computed by directly evaluating the basis functions on the given grid.
The computation of the pseudoinverse matrix as discussed above is outsourced to scipy's library.
This option can handle arbitrary grids and uses the full matrices for the transforms (i.e. you can still specify to throw out the less significant singular values in the singular value decomposition).
This makes `direct1` robust.
However, no simplifying assumptions are made, so it is likely to be the slowest.

The relavant code for this option builds the matrices exactly as discussed above.

To build the transform matrix for every combination of derivatives up to the given order:
```python
for d in self.derivatives:
    self._matrices["direct1"][d[0]][d[1]][d[2]] = self.basis.evaluate(
        self.grid.nodes, d, unique=True
    )
```
The `tranform(c)` method for a specified derivative combination:
```python
A = self.matrices["direct1"][dr][dt][dz]
return jnp.matmul(A, c)
```
To build the pseudoinverse:
```python
self._matrices["pinv"] = (
    scipy.linalg.pinv(A, rcond=rcond) if A.size else np.zeros_like(A.T)
)
```
The `fit(x)` method:
```python
Ainv = self.matrices["pinv"]
c = jnp.matmul(Ainv, x)
```

### Option 2: `direct2` nad option 3: `fft`
Functions of the toroidal coordinate $\zeta$ use Fourier series for their basis.
So a Fourier transform can be used to transform real space values to spectral space for the pseudoinverse matrix.

`Todo: Figure out how fft algorithm is used.`